## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,55.51,62,75,28.77,broken clouds
1,1,Hermanus,ZA,-34.4187,19.2345,77.81,77,7,6.82,clear sky
2,2,Sioux Lookout,CA,50.1001,-91.9170,-5.42,71,100,13.80,overcast clouds
3,3,Cape Town,ZA,-33.9258,18.4232,77.27,52,0,18.41,clear sky
4,4,Margate,GB,51.3813,1.3862,46.96,55,32,1.01,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hermanus,ZA,-34.4187,19.2345,77.81,77,7,6.82,clear sky
3,3,Cape Town,ZA,-33.9258,18.4232,77.27,52,0,18.41,clear sky
8,8,Port Alfred,ZA,-33.5906,26.8910,73.45,72,1,6.53,clear sky
9,9,Makakilo City,US,21.3469,-158.0858,63.61,83,0,4.61,clear sky
14,14,Mar Del Plata,AR,-38.0023,-57.5575,64.42,77,100,21.00,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
671,671,Baghdad,IQ,33.3406,44.4009,64.31,34,40,6.91,scattered clouds
678,678,Ishigaki,JP,24.3448,124.1572,69.31,79,100,19.13,overcast clouds
680,680,Mesetas,CO,3.3846,-74.0442,78.22,67,97,3.60,overcast clouds
684,684,Hwange,ZW,-18.3693,26.5019,77.83,69,90,5.44,light rain


In [6]:
# 4a. Determine if there are any empty rows.
if(clean_df.empty == True):
    print("no matching found. please try some input")
else:
    print("matching cities found")
  

matching cities found


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.dropna(inplace=True)

<ipython-input-7-6b442bd432d7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df.dropna(inplace=True)


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hermanus,ZA,77.81,clear sky,-34.4187,19.2345,
3,Cape Town,ZA,77.27,clear sky,-33.9258,18.4232,
8,Port Alfred,ZA,73.45,clear sky,-33.5906,26.8910,
9,Makakilo City,US,63.61,clear sky,21.3469,-158.0858,
14,Mar Del Plata,AR,64.42,overcast clouds,-38.0023,-57.5575,
20,Nador,MA,60.87,few clouds,35.1740,-2.9287,
21,Kinablangan,PH,76.59,broken clouds,7.6947,126.5503,
23,Acapulco,MX,75.02,broken clouds,16.8634,-99.8901,
24,Charters Towers,AU,72.99,clear sky,-20.1000,146.2667,
27,Kapaa,US,67.98,broken clouds,22.0752,-159.3190,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
        

In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace=True,subset=['Hotel Name'])

In [11]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))